背景：2023年10月13日微软成功收购动视暴雪，这是科技圈有史以来最大的收购案。从2022年1月18日宣布收购到最终完成交易历时21个月，无论是索尼的全力阻挠，还是美国、英国、欧盟三大战场的反垄断拉锯战，都对最终的结果产生了巨大的影响，我们希望分析股价数据，从而得出一些信息与趋势，并制定出自己的量化交易策略。

(1)人工筛选出50家不同行业的科技公司（包括ATVI和MSFT）， 通过Yahoo Finance API获得过去五年来每支股票的股价交易信息（包括开盘价、收盘价、交易量等），并储存在csv中。

In [3]:
import yfinance as yf
import pandas as pd

def get_stock_data(ticker, start_date, end_date):
    stock = yf.download(ticker, start=start_date, end=end_date)
    return stock

def save_to_csv(data, filename):
    data.to_csv(filename, index=True)

#五十家科技公司：软件、半导体、游戏、云服务、电商
symbol = {"SOFTWARE" : [
    "MSFT",
    "ADBE",
    "ORCL",
    "INTU",
    "SAP",
    "ADSK",
    "DOCU",
    "TEAM",
    "TWLO",
    "MDB"
],"GAMING" : [
    "ATVI",
    "EA",
    "TTWO",
    "NTDOY",
    "UBSFY",
    "HUYA",
    "NTES",
    "BILI",
    "SE",
    "NEXOF"
],"SEMIC" : [
    "NVDA",
    "AMD",
    "INTC",
    "QCOM",
    "AMAT",
    "AVGO",
    "TSM",
    "MU",
    "TXN",
    "ASML"
],"CLOUDC" : [
    "AMZN",
    "SNOW",
    "GOOGL",
    "CRM",
    "NOW",
    "VMW",
    "TWLO",
    "OKTA",
    "WDAY",
    "ZM"

],"E_CON" : [
    "PYPL",
    "BABA",
    "JD",
    "EBAY",
    "ETSY",
    "PDD",
    "MELI",
    "SHOP",
    "W",
    "OSTK"
]}

# 设置起始和结束日期
start_date = "2018-10-18"
end_date = "2023-10-18"

## 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

for sector, tickers in symbol.items():
    for ticker in tickers:
        stock_data = get_stock_data(ticker, start_date, end_date)
        stock_data["ticker"] = ticker  # 添加股票符号列
        stock_data["sector"] = sector  # 添加股票符号列
        all_data = pd.concat([all_data, stock_data], axis=0)

# 重新设置索引以包括日期列
all_data.reset_index(inplace=True)
# 保存数据到CSV文件
save_to_csv(all_data, "tech_stock_data.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

数据储存样式如下：

In [8]:
import yfinance as yf
import pandas as pd
data = pd.read_csv("tech_stock_data.csv", parse_dates=["Date"])
print(data)

       Unnamed: 0       Date        Open        High         Low       Close  \
0               0 2018-10-18  110.099998  110.529999  107.830002  108.500000   
1               1 2018-10-19  108.930000  110.860001  108.209999  108.660004   
2               2 2018-10-22  109.320000  110.540001  108.239998  109.629997   
3               3 2018-10-23  107.769997  108.970001  105.110001  108.099998   
4               4 2018-10-24  108.410004  108.489998  101.589996  102.320000   
...           ...        ...         ...         ...         ...         ...   
62241       62241 2023-10-11   16.680000   17.059999   16.073999   16.160000   
62242       62242 2023-10-12   16.090000   16.160000   14.990000   15.030000   
62243       62243 2023-10-13   14.910000   15.270000   14.860000   15.020000   
62244       62244 2023-10-16   15.030000   17.549999   14.680000   17.250000   
62245       62245 2023-10-17   17.389999   18.090000   16.959999   17.879999   

        Adj Close    Volume ticker    s

（2）对于每个行业内的十家公司，计算从 "2018-10-18"-"2023-10-18"每支股票的单日回报$\frac{今日收盘价−昨日收盘价}{昨日收盘价}$、累计回报、$年化回报=1+\frac{累计回报}{100}^{\frac{1}{投资年限}}-1$

In [9]:
# 根据行业划分数据
sectors = data['sector'].unique()
# 创建一个空字典以存储按行业划分的结果
sector_results = {}

# 分析每个行业
for sector in sectors:
    # 根据行业筛选数据
    sector_data = data[data['sector'] == sector].copy()
    # 获取行业内的所有股票
    tickers = sector_data['ticker'].unique()
    # 创建一个空字典以存储每支股票的结果
    ticker_results = {}
    # 分别分析每支股票
    for ticker in tickers:
        # 根据股票筛选数据
        stock_data = sector_data[sector_data['ticker'] == ticker].copy()
        # 计算日回报
        stock_data['Daily_Return'] = stock_data['Close'].pct_change()
        # 计算累积回报
        stock_data['Cumulative_Return'] = (1 + stock_data['Daily_Return']).cumprod() - 1
        #年化收益
        last_day_cumulative_return = stock_data['Cumulative_Return'].iloc[-1]
        years = (stock_data['Date'].max() - stock_data['Date'].min()).days / 365.25
        annualized_return = (1 + last_day_cumulative_return) ** (1 / years) - 1
        # 在股票字典中存储结果
        ticker_results[ticker] = {
            '累积回报': last_day_cumulative_return,
            '年化回报': annualized_return
        }

    # 在行业字典中存储每支股票的结果
    sector_results[sector] = ticker_results

# 显示结果
for sector, ticker_results in sector_results.items():
    for ticker, result in ticker_results.items():
        print(f"行业: {sector}")
        print(f"股票: {ticker}")
        print(f"累积回报: {result['累积回报']:.4f}")
        print(f"年化回报: {result['年化回报']:.4f}")
        print("\n")


行业: SOFTWARE
股票: MSFT
累积回报: 2.0605
年化回报: 0.2509


行业: SOFTWARE
股票: ADBE
累积回报: 1.2372
年化回报: 0.1749


行业: SOFTWARE
股票: ORCL
累积回报: 1.3014
年化回报: 0.1815


行业: SOFTWARE
股票: INTU
累积回报: 1.5333
年化回报: 0.2045


行业: SOFTWARE
股票: SAP
累积回报: 0.2194
年化回报: 0.0405


行业: SOFTWARE
股票: ADSK
累积回报: 0.5587
年化回报: 0.0929


行业: SOFTWARE
股票: DOCU
累积回报: -0.0813
年化回报: -0.0168


行业: SOFTWARE
股票: TEAM
累积回报: 1.4153
年化回报: 0.1930


行业: SOFTWARE
股票: TWLO
累积回报: -0.1896
年化回报: -0.0412


行业: SOFTWARE
股票: MDB
累积回报: 4.3162
年化回报: 0.3971


行业: GAMING
股票: ATVI
累积回报: 0.3149
年化回报: 0.0563


行业: GAMING
股票: EA
累积回报: 0.2385
年化回报: 0.0437


行业: GAMING
股票: TTWO
累积回报: 0.1418
年化回报: 0.0269


行业: GAMING
股票: NTDOY
累积回报: 0.1708
年化回报: 0.0321


行业: GAMING
股票: UBSFY
累积回报: -0.7088
年化回报: -0.2188


行业: GAMING
股票: HUYA
累积回报: -0.8493
年化回报: -0.3153


行业: GAMING
股票: NTES
累积回报: 1.4476
年化回报: 0.1962


行业: GAMING
股票: BILI
累积回报: 0.0658
年化回报: 0.0128


行业: GAMING
股票: SE
累积回报: 2.5398
年化回报: 0.2879


行业: GAMING
股票: NEXOF
累积回报: 0.3301
年化回报: 0.0588


行业: SEMIC
股票: N

对每个行业的公司的年化收益进行排序，见输出结果。

In [10]:
import pandas as pd

# 存储按行业划分的结果的数据框
sector_results_df = pd.DataFrame(columns=['行业', '股票', '累积回报', '年化回报'])

# 将数据转化为dataframe
for sector, ticker_results in sector_results.items():
    for ticker, result in ticker_results.items():
        # 存储按行业划分的结果
        sector_results_df.loc[len(sector_results_df)] = {
            '行业': sector,
            '股票': ticker,
            '累积回报': result['累积回报'],
            '年化回报': result['年化回报']
        }

#按照年化收益排序
sorted_sector_results_df = pd.DataFrame(columns=['行业', '股票', '累积回报', '年化回报'])
for sector, group in sector_results_df.groupby('行业'):
    sorted_group = group.sort_values(by='年化回报', ascending=False)
    sorted_sector_results_df = pd.concat([sorted_sector_results_df, sorted_group])

print("按行业划分的结果数据框（已排序）：")
# print(sorted_sector_results_df)
# 转换DataFrame为Markdown格式
markdown_table = sorted_sector_results_df.to_markdown()
print(markdown_table)

按行业划分的结果数据框（已排序）：


C:\Users\12040\AppData\Local\Temp\ipykernel_34184\1401273582.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_sector_results_df = pd.concat([sorted_sector_results_df, sorted_group])


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

对上述数据进行可视化，用柱状图能够清晰的看出每个行业各个公司的排名情况。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 根据行业划分数据框
sectors = sorted_sector_results_df['行业'].unique()
# 遍历每个行业，生成可视化图表
for sector in sectors:
    # 获取当前行业的数据
    sector_data = sorted_sector_results_df[sorted_sector_results_df['行业'] == sector]
    # 创建子图
    fig, axes = plt.subplots(figsize=(10, 5))
    # 行业内各股票的年化回报率柱状图
    sns.barplot(ax=axes, x='年化回报', y='股票', data=sector_data)
    axes.set_title(f'Annualized Return of Stocks in the {sector} Sector')
    axes.set_xlabel('Annualized Return')
    axes.set_ylabel('Stock')
    # 显示图表
    plt.show()


首先以收盘价为数据，画出每个行业三年来各公司的股价趋势图，对数据有一个直观印象。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

filtered_data = data[data['Date'] >= '2021-10-18']
# 设置图形大小和样式
plt.figure(figsize=(15, 20))  # 调整图形大小
sns.set(style="darkgrid")
# 遍历每个行业，为每个行业创建一个图
sectors = filtered_data['sector'].unique()

# 遍历每个行业
for i, sector in enumerate(sectors, start=1):
    sector_data = filtered_data[filtered_data['sector'] == sector]
    # 创建子图
    plt.subplot(len(sectors), 1, i)
    # 遍历每支股票
    for ticker in sector_data['ticker'].unique():
        #突出显示两支股票
        if ticker == "MSFT"or ticker =="ATVI":
            linewide = 4
        else:
            linewide = 2
    
        stock_data = sector_data[sector_data['ticker'] == ticker]

        # 绘制股价趋势
        plt.plot(stock_data['Date'], stock_data['Close'], label=ticker,linewidth = linewide)
    # 添加 MSFT 和 ATVI 的股价趋势（如果不存在的话）
    if 'MSFT' not in sector_data['ticker'].unique():
        msft_data = filtered_data[filtered_data['ticker'] == 'MSFT']
        plt.plot(msft_data['Date'], msft_data['Close'], label='MSFT', linewidth=4)
    if 'ATVI' not in sector_data['ticker'].unique():
        atvi_data =  filtered_data[filtered_data['ticker'] == 'ATVI']
        plt.plot(atvi_data['Date'], atvi_data['Close'], label='ATVI', linewidth=4)
    plt.title(f'{sector} trend')
    plt.xlabel('day')
    plt.ylabel('close prize')
    plt.legend()

# 调整布局以获得更好的可见性
plt.tight_layout()
plt.show()

首先以ATVI股票数据为例进行分析。
（1）画出股票价格与RSI图像。
相对强弱指数（RSI）是一种常用的动量指标，用于度量一定时期内价格上涨和下跌的速度和幅度。RSI的取值范围在0到100之间，通常用来判断市场是否处于超买或超卖状态。RSI的一般规则是，当RSI超过70时，市场被视为超买（overbought），可能发生价格下跌的反转；当RSI低于30时，市场被视为超卖，可能发生价格上涨的反转。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取股票数据
atvi_data = data[data['ticker'] == 'ATVI']
atvi_data = atvi_data[atvi_data['Date'] >= '2021-05-01']

# 计算价格变动
atvi_data['Price Gain'] = atvi_data['Close'].diff()

# 计算平均涨幅和跌幅
atvi_data['Average Gain'] = atvi_data['Price Gain'].clip(lower=0).rolling(window=14).mean()
atvi_data['Average Loss'] = -atvi_data['Price Gain'].clip(upper=0).rolling(window=14).mean()

# 计算相对强弱指数 (RSI)
atvi_data['RS'] = atvi_data['Average Gain'] / atvi_data['Average Loss']
atvi_data['RSI'] = 100 - (100 / (1 + atvi_data['RS']))

# 绘制股价和RSI图表
fig, ax1 = plt.subplots(figsize=(12, 6))

# 绘制股价
ax1.plot(atvi_data['Date'], atvi_data['Close'], label='ATVI Close Price', color='b')
ax1.set_xlabel('Date')
ax1.set_ylabel('Close Price', color='b')
ax1.tick_params('y', colors='b')

# 创建第二个y轴用于绘制RSI
ax2 = ax1.twinx()
ax2.plot(atvi_data['Date'], atvi_data['RSI'], label='RSI', color='r')
ax2.set_ylabel('RSI', color='r')
ax2.tick_params('y', colors='r')

plt.title('ATVI Stock Price and RSI')
plt.show()

（3）为了识别长期趋势，我们采用移动平均的方法来处理时间序列数据来消除短期的波动性，从而识别长期的趋势。这里我们使用指数加权移动平均（赋予最近数据更多的权重）的方法来处理数据，见图中的橙色曲线，我们可以通过曲线的转折点来判断趋势，例如2022年8月，各国开启反垄断调查，股价呈现出明显的下降趋势。同时我们还画出了每天的对应交易量数据,能够反应市场的活跃度并确认价格趋势的可靠性.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# 筛选出ATVI股票的数据
atvi_data = data[data['ticker'] == 'ATVI']

# 筛选出近两年的数据
atvi_data = atvi_data[atvi_data['Date'] >= '2021-05-01']

# 20日指数移动平均线：消除短期波动性，识别长期趋势
atvi_data['EMA_20'] = atvi_data['Close'].ewm(span=20, adjust=False).mean()
atvi_data['SMA_20'] = atvi_data['Close'].rolling(window=20).mean()

# 创建画布
fig, ax1 = plt.subplots(figsize=(12, 6))

# 绘制股价和平均移动线的趋势图
ax1.plot(atvi_data['Date'], atvi_data['Close'], label='ATVI prize')
ax1.plot(atvi_data['Date'], atvi_data['EMA_20'], label='20-day EMA')
ax1.set_title('ATVI Price and 20-day EMA and Volume')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price')
ax1.legend(loc='upper left')

# 在指定位置添加箭头
arrow_date = '2022-08-23'  # 箭头的日期
arrow_date_num = mdates.datestr2num(arrow_date)  # 将日期字符串转换为数字
arrow_price = atvi_data[atvi_data['Date'] == arrow_date]['Close']  # 箭头的价格
arrow_text = 'start a downward trend'  # 箭头的文本
arrow_text_position = (mdates.datestr2num('2022-10-01'), arrow_price + 3)
ax1.annotate(arrow_text, xy=(arrow_date_num, arrow_price), xytext=arrow_text_position,
             arrowprops=dict(facecolor='red', shrink=0.05,connectionstyle='arc3,rad=0.2'),rotation_mode='anchor',
             fontsize=10, color='black', ha='left', va='bottom')


# 创建第二个坐标轴，共享X轴
ax2 = ax1.twinx()

# 绘制交易量图
ax2.plot(atvi_data['Date'], atvi_data['Volume'], alpha=0.5, color='purple', label='Volume')
ax2.set_ylabel('Volume')
ax2.legend(loc='upper right')
plt.show()



此外我们还可以计算出daily return较大的点来找出股价剧烈变动的时间节点,例如22-01-18,微软宣布收购动视暴雪后股价激增,2023-04-26英国反垄断机构CMA不通过股价暴跌,以及2023-07-11美国反垄断机构通过.见question5代码

我们计算各支股票与ATVI与MSFT的相关系数,并选出相关系数较高的六支股票

In [ ]:

#找出相关系数最高的十只股票
import pandas as pd
# 筛选出 ATVI 和 MSFT 的数据
atvi_data = data[data['ticker'] == 'ATVI'].reset_index(drop=True)
msft_data = data[data['ticker'] == 'MSFT'].reset_index(drop=True)

tickers = data['ticker'].unique()
correlation_with_atvi = {}
correlation_with_msft = {}

for ticker in tickers:
    if ticker not in ['ATVI', 'MSFT']:
        stock_data = data[data['ticker'] == ticker].reset_index(drop=True)
        correlation_with_atvi[ticker] = atvi_data['Close'].corr(stock_data['Close'])
        correlation_with_msft[ticker] = msft_data['Close'].corr(stock_data['Close'])

# 将结果转换为 DataFrame
result_df_atvi = pd.DataFrame(list(correlation_with_atvi.items()), columns=['Ticker', 'Correlation_with_ATVI'])
result_df_msft = pd.DataFrame(list(correlation_with_msft.items()), columns=['Ticker', 'Correlation_with_MSFT'])

top_5_atvi = result_df_atvi.nlargest(5, 'Correlation_with_ATVI')
top_5_msft = result_df_msft.nlargest(5, 'Correlation_with_MSFT')
print(atvi_data['Close'].corr(msft_data['Close']))
# 打印前五个
print(top_5_atvi)
print(top_5_msft)
eight_stock = ["ATVI","MSFT", "NTES" ,"EA","TSM","GOOGL","AMD" ,"ASML" ]

以ATVI和MSFT为基准,画出对比公司的股票数据信息,side by side

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 选取八只股票
eight_stocks = ["ATVI", "MSFT", "NTES", "GOOGL", "EA","AMD", "TSM", "ASML"]

# 创建画布
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(16, 16))
fig.suptitle('Stock Price and Volume Trend for Eight Stocks', fontsize=20)

# 遍历八只股票并在子图上绘制趋势图
for i, stock_ticker in enumerate(eight_stocks):
    # 获取股票数据
    stock_data = data[data['ticker'] == stock_ticker]
    stock_data = stock_data[stock_data['Date'] >= '2020-01-01']

    # 20日移动平均线
    stock_data['EMA_20'] = stock_data['Close'].ewm(span=20, adjust=False).mean()

    # 子图索引
    row = i // 2
    col = i % 2

    # 绘制趋势图
    axes[row, col].plot(stock_data['Date'], stock_data['Close'], label='Price')
    axes[row, col].plot(stock_data['Date'], stock_data['EMA_20'], label='20-day EMA')
    axes[row, col].set_title(f'{stock_ticker} Price and 20-day EMA')
    axes[row, col].set_xlabel('Date')
    axes[row, col].set_ylabel('Price')
    axes[row, col].legend(loc='upper left')

    # 创建第二个坐标轴，绘制交易量图
    ax2 = axes[row, col].twinx()
    ax2.plot(stock_data['Date'], stock_data['Volume'], alpha=0.5, color='purple', label='Volume')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper right')

# 调整子图之间的间距
fig.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()
